### P3 Flow Setup

## read CSV

In [1]:
import csv
import cv2
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split


CSV_PATH = "./data/driving_log.csv"

samples = []
with open(CSV_PATH) as csvfile:
    next(csvfile, None)
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
    
print("total sample count: ", len(samples))

train_samples, validation_samples = train_test_split(samples, test_size = 0.2)

print("train sample count:", len(train_samples), " validation sample count: ", len(validation_samples))

print(train_samples[0])

total sample count:  8036
train sample count: 6428  validation sample count:  1608
['IMG/center_2016_12_01_13_41_47_700.jpg', ' IMG/left_2016_12_01_13_41_47_700.jpg', ' IMG/right_2016_12_01_13_41_47_700.jpg', ' -0.2686331', ' 0.9855326', ' 0', ' 30.18304']


In [2]:
from sklearn.utils import shuffle

IMG_PATH_PREFIX = "./data/IMG/"

def generator(samples, batch_size=64):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = IMG_PATH_PREFIX  + batch_sample[0].split('/')[-1]
                
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                
                #center_image = np.expand_dims(center_image, axis=0)
                #center_angle = np.expand_dims(center_angle, axis=0)
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            
            #X_train = np.expand_dims(X_train, axis=0)
            #y_train = np.expand_dims(y_train, axis=1)
            print("X_train shape: ", X_train.shape, " y_train shape:", y_train.shape)
            #print("X train: ", X_train)
            yield X_train, y_train


train_generator = generator(train_samples, batch_size = 32)
validation_generator = generator(validation_samples, batch_size = 32)
            
print("generator output: \n", next(train_generator))
    

X_train shape:  (32, 160, 320, 3)  y_train shape: (32,)
generator output: 
 (array([[[[233, 180, 147],
         [253, 200, 167],
         [226, 173, 140],
         ..., 
         [178, 138, 110],
         [178, 138, 110],
         [178, 138, 110]],

        [[101,  48,  15],
         [234, 181, 148],
         [245, 192, 159],
         ..., 
         [178, 138, 110],
         [178, 138, 110],
         [178, 138, 110]],

        [[240, 187, 154],
         [246, 193, 160],
         [235, 182, 149],
         ..., 
         [179, 139, 111],
         [179, 139, 111],
         [179, 139, 111]],

        ..., 
        [[125, 146, 148],
         [121, 142, 144],
         [119, 140, 142],
         ..., 
         [ 73,  87,  86],
         [ 99, 113, 112],
         [123, 137, 136]],

        [[118, 139, 141],
         [119, 140, 142],
         [119, 140, 142],
         ..., 
         [ 59,  73,  72],
         [ 63,  77,  76],
         [ 72,  86,  85]],

        [[111, 132, 134],
         [116, 137

In [4]:
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers.convolutional import Cropping2D, Convolution2D, MaxPooling2D
from keras.layers.core import Lambda, Dense, Activation, Flatten

image_color_channel = 3
image_height = 160
image_width = 320

crop_top = 50
crop_bottom = 20
crop_left = 0
crop_right = 0

model = Sequential()

#cropping layer
model.add(Cropping2D(cropping=((50,20), (1,1)), input_shape=(160,320,3),  dim_ordering='tf'))
#model.add(Dense(1))
#model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))


#normalized layer
#model.add(Lambda(lambda x: x / 127.5 - 1.))

#conv layar
#model.add(Convolution2D(24, 5, 5,  border_mode="valid"))

#output layer
#model.add(Dense(1))


model.compile(loss = "mse", optimizer="adam")
print(model.summary())
model.fit_generator(train_generator, samples_per_epoch= len(train_samples), validation_data=validation_generator, nb_val_samples=len(validation_samples), nb_epoch=3)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_2 (Cropping2D)        (None, 90, 318, 3)    0           cropping2d_input_2[0][0]         
Total params: 0
Trainable params: 0
Non-trainable params: 0
____________________________________________________________________________________________________
None
Epoch 1/3
X_train shape:  (32, 160, 320, 3)  y_train shape: (32,)
X_train shape:  (32, 160, 320, 3)  y_train shape: (32,)


ValueError: Error when checking model target: expected cropping2d_2 to have 4 dimensions, but got array with shape (32, 1)